In [1]:
!pip install -U -q scikit-learn
!pip install -q numpy
!pip install -q dask-ml
!pip install -q hyperopt
!pip install -q optuna
!pip install -q dask
!pip install -q tensorflow
!pip intall -q keras


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Grid Search

### Self Implementation

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [3]:
iris = load_iris()
X, y = iris.data, iris.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

### We are using Support vector machine to train a classifier

Checkout more hyperparameters for SVC here: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

Define more hyperparameters in `param_grid` and play around with search space.

In [5]:
# Defining Search Space

# Input your code here to play around with Search Spaces
# define more hyperparameters

c = 0.001
gamma = 1e-10

param_grid = {
              "C": [c*(10**i) for i in range(1,14)],
              "gamma": [gamma*(10**i) for i in range(1,14)]
             }

In [6]:
from itertools import product

def make_sets(grid):
    """function makes all possible set from the grid above"""
    sets = list()
    all_hps_vals = [lst for lst in param_grid.values()]
    hp_keys = [hp for hp in param_grid.keys()]
    val_sets = product(*all_hps_vals)
    for val in val_sets:
        hp_set = dict()
        for idx, hp_key in enumerate(hp_keys):
            hp_set[hp_key] = val[idx]
        sets.append(hp_set)
    return sets

make_sets(param_grid)[:5]

[{'C': 0.01, 'gamma': 1e-09},
 {'C': 0.01, 'gamma': 1e-08},
 {'C': 0.01, 'gamma': 1.0000000000000001e-07},
 {'C': 0.01, 'gamma': 1e-06},
 {'C': 0.01, 'gamma': 1e-05}]

In [7]:
def grid_search(clf, grid, X_train, y_train, X_test, y_test):
    # iterates over all the sets
    all_sets = make_sets(grid)
    logs = list()
    best_hp_set = {
        "best_test_score": 0.0
    }
    for hp_set in all_sets:
        log = dict()
        model = clf(**hp_set)
        model.fit(X_train, y_train)
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)
        
        log["hp"] = hp_set
        log["train_score"] = train_score
        log["test_score"] = test_score
        
        if best_hp_set["best_test_score"]<test_score:
            best_hp_set["best_test_score"] = test_score
            best_hp_set["hp_set"] = hp_set
        
        logs.append(log)
        
    return logs, best_hp_set

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [9]:
logs, best = grid_search(SVC, param_grid, X_train, y_train, X_test, y_test)

In [10]:
# print(best)

### Scikit Learn Implementaion

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
# using k fold cross validation, here k=3

clf = GridSearchCV(SVC(), param_grid, cv=3)
clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0,
                               100000.0, 1000000.0, 10000000.0, 100000000.0,
                               1000000000.0, 10000000000.0],
                         'gamma': [1e-09, 1e-08, 1.0000000000000001e-07, 1e-06,
                                   1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0,
                                   100.0, 1000.0]})

In [13]:
clf.best_estimator_

SVC(C=100.0, gamma=0.01)

In [14]:
# print(f'Test Score: {clf.score(X_test, y_test)}')
# print(f'Train Score: {clf.score(X_train, y_train)}')

## Random Search

### Self Implementation

#### Change values here in `param_grid` to play with search space

In [15]:
from sklearn.svm import SVC


In [16]:
import random
import numpy as np

def loguniform(low=0, high=1, size=100, base=10):
    """function creates a log uniform distribution with 
    random values."""
    return np.power(base, np.random.uniform(low, high, size))

param_grid = {
              "gamma": loguniform(low=-10, high=4, base=10),
              "C": loguniform(low=-3, high=11, base=10)
             }

In [17]:
def get_random_hp_set(grid):
      # function chooses a random value for each from grid
    hp_set = dict()
    for key, param in grid.items():
             hp_set[key] = np.random.choice(param) 
    return hp_set

In [18]:
def random_search(clf, grid, n_iterations, X_train, y_train, X_test, y_test):
# defining function for random search    
    logs = list()
    best_hp_set = {
    "best_test_score": 0.0
    }

    for iteration in range(n_iterations):
        log = dict()

        # selecting the set of hyperparameters from function defined
        # for random search.
        hp_set = get_random_hp_set(grid)
        # print(hp_set)
        model = clf(**hp_set)
        model.fit(X_train, y_train)
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)

        log["hp"] = hp_set
        log["train_score"] = train_score
        log["test_score"] = test_score

        if best_hp_set["best_test_score"]<test_score:
            best_hp_set["best_test_score"] = test_score
            best_hp_set["hp_set"] = hp_set

        logs.append(log)

    return logs, best_hp_set

In [19]:
logs, best = random_search(SVC, param_grid, 20, X_train, y_train, X_test, y_test)

In [20]:
best

{'best_test_score': 1.0,
 'hp_set': {'gamma': 0.00030548136524084335, 'C': 21903676817.243515}}

### Scikit Learn Implementation

In [21]:
from sklearn.model_selection import RandomizedSearchCV

In [22]:
clf = RandomizedSearchCV(SVC(), param_grid, n_iter=20, cv=3)
clf.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=20,
                   param_distributions={'C': array([4.83397523e-03, 3.06630664e+02, 2.46192207e+08, 2.01473719e-01,
       3.13042218e-03, 5.56258362e-01, 7.57056937e-01, 4.81966514e-01,
       7.40446282e+07, 1.75830199e+04, 5.01719159e+00, 9.30305302e-03,
       6.29616216e+07, 2.19036768e+10, 7.21121982e+01, 1.68929189e-03,
       1.67410938e-02, 2.59492012e+0...
       1.55740120e+01, 3.02154412e-06, 1.75542806e-06, 1.98368741e+02,
       1.56974424e-09, 1.04703916e+01, 1.54939163e-07, 1.70868633e+02,
       1.69117291e-08, 5.72949892e-01, 8.16116815e+03, 2.97444425e-10,
       6.18537711e-01, 2.20637034e-04, 5.17878498e+00, 1.15924198e-10,
       1.07525300e+02, 4.14621758e-09, 1.88757996e-08, 1.90660972e-09,
       2.59764108e+02, 1.53312507e-10, 5.75892917e-10, 2.51081155e-03])})

In [23]:
clf.best_estimator_

SVC(C=336163088.1545798, gamma=3.13145092305502e-08)

In [24]:
# print(f'Test Score: {clf.score(X_test, y_test)}')
# print(f'Train Score: {clf.score(X_train, y_train)}')

## Dask

### Initialize the client first

### Define client such that you don't hang your system

In [25]:
from dask.distributed import Client
client = Client(processes=False, threads_per_worker=4, memory_limit='4GB', n_workers=1)

/home/tanay/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [26]:
client

Client Scheduler: inproc://192.168.0.36/72345/1 Dashboard: http://192.168.0.36:8787/status,Cluster Workers: 1 Cores: 4 Memory: 4.00 GB


#### Use `client.dashboard_link` to monitor the distribution over your system cores

In [27]:
client.dashboard_link

'http://192.168.0.36:8787/status'

### Now we'll distribute a huge dataset, using Dask package

### Let's simply train a model first

In [28]:
from dask_ml import datasets
from dask_ml.model_selection import train_test_split
import dask.array as da
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier

/home/tanay/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/tanay/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/tanay/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [29]:
X, y = datasets.make_classification(n_samples=1000000,
                                         n_features=7,
                                         random_state=0,
                                         chunks=100000)

In [30]:
X

dask.array<normal, shape=(1000000, 7), dtype=float64, chunksize=(100000, 7), chunktype=numpy.ndarray>

In [31]:
y

dask.array<astype, shape=(1000000,), dtype=int64, chunksize=(100000,), chunktype=numpy.ndarray>

In [32]:
classes = da.unique(y).compute()

In [33]:
classes

array([0, 1])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

#### Note that we can only use modeling algorithms which supports Batch Training, to train model in batches

In [35]:
clf = SGDClassifier(loss='log', penalty='l2', tol=0.01)
# wrapping in Incremental
clf = Incremental(clf, scoring='accuracy')
clf.fit(X_train, y_train, classes=classes)
# while training check Client Dashboard

/home/tanay/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Incremental(estimator=SGDClassifier(loss='log', tol=0.01), scoring='accuracy')

### For algorithms like SVC we can not train data in batches.

### But hyperparameter Optimization can be distributed, let's see how

In [36]:
from sklearn.datasets import load_digits
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
import joblib
import time

In [37]:
# define a simple classifier with GridSearch

X, y = load_digits().data, load_digits().target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle=True)

c = 0.001
gamma = 1e-10
param_grid = {
              "C": [c*(10**i) for i in range(1,14)],
              "gamma": [gamma*(10**i) for i in range(1,14)]
             }

clf = SVC(kernel='rbf')
search = GridSearchCV(clf, param_grid, cv=3)

#### **monitor dashboard after executing this next piece of code, and see how grid search is distributed over different cores**

In [38]:
since = time.time()
with joblib.parallel_backend('dask', scatter=[X_train, y_train]):
    model = search.fit(X_train, y_train)
print(time.time()-since)

19.800275325775146


### Something like this would not work! Why?

### Unfortunately for a dask dataset, you cannot use both, an algorithm which train in batches and optimize hyperparameters as well

In [39]:
# from dask_ml import datasets
# from dask_ml.wrappers import Incremental
# from dask_ml.model_selection import train_test_split, GridSearchCV
# from dask_ml.metrics import accuracy_score

# from sklearn.metrics import make_scorer
# from sklearn.linear_model import SGDClassifier

# import joblib

# import dask.array as da
# from dask.distributed import Client
# client = Client(processes=False)

# param_grid = {
#               "loss": ['hinge', 'log'],
#               "tol": [1e-2, 1e-3]
#              }

# X, y = datasets.make_classification(n_samples=100000,
#                                     n_features=7,
#                                     random_state=0,
#                                     chunks=10000)

# # providing an accuracy metrics from 'dask_ml'
# scorer = make_scorer(accuracy_score)

# X_train, X_test, y_train, y_test = train_test_split(X, y)

# clf = SGDClassifier()
# clf_wrap = Incremental(clf, scoring=scorer)
# searh_clf = GridSearchCV(clf_wrap, param_grid, cv=3)

# with joblib.parallel_backend('dask'):
#     model = searh_clf.fit(X_train, y_train)

## HyperOpt

### A basic where we are optimizing, f(a,b) = a\*\*2 - b\*\*2

In [40]:
from hyperopt import tpe, fmin, hp

def objective_func(args):
    a = args['a']
    b = args['b']    
    f = a**2 - b**2
    return f

range_a = hp.uniform('a', -2, 3)
range_b = hp.uniform('b', -1, 2)

space = {'a': range_a,
            'b': range_b}

best = fmin(objective_func, space, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [00:00<00:00, 321.72trial/s, best loss: -3.703622010557403]


In [41]:
best

{'a': -0.4843757303942593, 'b': 1.9845004053293553}

### Using Hyperopt to find algorithm and hyperparameters

In [42]:
from hyperopt import tpe, fmin, hp
import math as m

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [43]:
space = hp.choice('classifier',[
        {'model': 'KNeighborsClassifier',
        'param': {'n_neighbors':
                        hp.choice('n_neighbors',range(3,11)),
        'algorithm':hp.choice('algorithm', ['ball_tree', 'kd_tree']),
        'leaf_size':hp.choice('leaf_size', range(1,50)),
        'metric':hp.choice('metric', ["euclidean", "manhattan",
                           "chebyshev", "minkowski"
                           ])}
        },
        {'model': 'SVC',
        'param':{'C':hp.loguniform('C', -2*m.log(10), 11*m.log(10)),
        'kernel':hp.choice('kernel',['rbf', 'poly', 'sigmoid']),
        'degree':hp.choice('degree', range(1,6)),
        'gamma':hp.loguniform('gamma', -9*m.log(10), 3*m.log(10))}
        }
        ])

In [44]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data,
                                                    digits.target,
                                                    test_size=0.3)

logs = {'args':list(),
        'train_score': list(),
        'val_score': list()}

def objective_func(args):
    clf_func = args["model"]
    params = args["param"]
    
    # debugging with hyperopt is a pain, so be smart and use these statements when stuck
#     print(args)
    clf = eval(clf_func)(**params)
    clf.fit(X_train, y_train)
    
    val_score = clf.score(X_test, y_test)
    train_score = clf.score(X_train, y_train)
    
    logs['args'].append(args)
    logs['train_score'].append(train_score)
    logs['val_score'].append(val_score)
    
    return -val_score

best = fmin(objective_func, space, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [00:16<00:00,  6.15trial/s, best loss: -0.9981481481481481]


In [45]:
best

{'C': 104313200.80236481,
 'classifier': 1,
 'degree': 4,
 'gamma': 7.341558334446171e-05,
 'kernel': 0}

### Let's try and optimize a Neural Network

In [46]:
from hyperopt import hp, tpe, fmin
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
import numpy as np

# load and preprocess the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
classes = 10
input_shape = 784
y_train = np_utils.to_categorical(y_train, classes)
y_test = np_utils.to_categorical(y_test, classes)

#logs
logs = {'model_summary':list(),
        'val_acc': list()}

def obj_func(args):
    
    model = Sequential()
    
    #defining first hidden layer
    model.add(Dense(units=args['units']['layer_units_1'], 
                    input_shape=(input_shape, ),
                    name='layer_units_1'))
    
    #defining number of remaining hidden layer
    number_of_layers = len(args['units'])
    for layer in range(2, number_of_layers):
        model.add(Dense(units=args['units'][f'layer_units_{layer}'], 
                        name=f'layer_units_{layer}'))
        model.add(Dropout(args['dropout'][f'dropout_p_{layer}'], 
                          name=f'dropout_p_{layer}'))
        model.add(Activation(activation=args['activation'][f'activation_{layer}'], 
                             name=f'activation_{layer}'))
        
    model.add(Dense(classes, name=f'layer_unit_{layer+1}'))
    model.add(Activation(activation='softmax', name=f'activation_{layer+1}'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer='adam')
    
    result = model.fit(x_train, y_train,
                      batch_size=2,
                      epochs=1,
                      verbose=3,
                      validation_split=0.2)
    
    validation_acc = np.amax(result.history['val_accuracy'])
    print(validation_acc)
    logs['model_summary'].append(model.summary())
    logs['val_acc'].append(validation_acc)
    
    return -validation_acc

def each_layer(number_of_layers):
    params = {'units': dict(),
              'dropout': dict(),
              'activation': dict()}
    number_of_nodes = [16,36,64,128,256,512]
    for layer in range(number_of_layers):
        params['units'][f'layer_units_{layer}'] = hp.choice(f'layer_{number_of_layers}_{layer}', 
                                                            number_of_nodes)
        params['dropout'][f'dropout_p_{layer}'] = hp.uniform(f'dropout_{number_of_layers}_{layer}', 
                                                             0, 
                                                             0.8)
        params['activation'][f'activation_{layer}'] = hp.choice(f'activation_{number_of_layers}_{layer}', 
                                                                ['relu', 'elu'])
    return params

number_of_layers = [3, 5, 7, 9]
space = hp.choice('layers', [each_layer(n) for n in number_of_layers])
best = fmin(obj_func, space, algo=tpe.suggest, max_evals=5)

0.8913333415985107                                   
Model: "sequential"                                  

_________________________________________________________________

 Layer (type)                Output Shape              Param #   


 layer_units_1 (Dense)       (None, 64)                50240     

 layer_units_2 (Dense)       (None, 128)               8320      

 dropout_p_2 (Dropout)       (None, 128)               0         

 activation_2 (Activation)   (None, 128)               0         

 layer_units_3 (Dense)       (None, 64)                8256      

 dropout_p_3 (Dropout)       (None, 64)                0         

 activation_3 (Activation)   (None, 64)                0         

 layer_units_4 (Dense)       (None, 64)                4160      

 dropout_p_4 (Dropout)       (None, 64)                0         

 activation_4 (Activation)   (None, 64)                0         

 layer_unit_5 (Dense)        (None, 10)                650       

 activation_5 (Acti

In [47]:
best

{'activation_5_0': 1,
 'activation_5_1': 0,
 'activation_5_2': 1,
 'activation_5_3': 0,
 'activation_5_4': 1,
 'dropout_5_0': 0.1783234965278652,
 'dropout_5_1': 0.27846772605080233,
 'dropout_5_2': 0.6416050660560999,
 'dropout_5_3': 0.3588056762389913,
 'dropout_5_4': 0.004524594522497249,
 'layer_5_0': 1,
 'layer_5_1': 2,
 'layer_5_2': 3,
 'layer_5_3': 2,
 'layer_5_4': 2,
 'layers': 1}

### number of trials are too low here, but we can definitly perform an architecture search using this by increasing the number of trials

### Now go ahead and explore this amazing wrapper `Hyperas`

## Optuna

### Let's train a scikit-learn model with this library

In [48]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import optuna
from optuna.samplers import TPESampler

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data,
                                                    digits.target,
                                                    test_size=0.3)

def log(study, trial):
    print(f"Trial No.={trial.number}, HP_Set={trial.params}, Score={trial.value}")
    print(f"Best Value ={study.best_value}")

def objective_func(trial):
    
    classifier_name = trial.suggest_categorical("classifier", ["SVC", "RandomForest"])
    if classifier_name == "SVC":
        c = trial.suggest_loguniform("svc_c", 1e-2, 1e+11)
        gamma = trial.suggest_loguniform("svc_gamma", 1e-9, 1e+3)
        kernel = trial.suggest_categorical("svc_kernel", ['rbf','poly','rbf','sigmoid'])
        degree = trial.suggest_categorical("svc_degree", range(1,15))
        clf = SVC(C=c, gamma=gamma, kernel=kernel, degree=degree)
    else:
        algorithm = trial.suggest_categorical("algorithm", ['ball_tree', "kd_tree"])
        leaf_size = trial.suggest_categorical("leaf_size", range(1,50))
        metric = trial.suggest_categorical("metic", ["euclidean","manhattan", "chebyshev","minkowski"])
        clf = KNeighborsClassifier(algorithm=algorithm, leaf_size=leaf_size, metric=metric)
        
    clf.fit(X_train, y_train)
    val_acc = clf.score(X_test, y_test)
    
    return val_acc

study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective_func, n_trials=3, callbacks=[log])
best_trial = study.best_trial.value

print(f"Best trial  accuracy: {best_trial}")
print("parameters for best trail are :")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

[I 2023-06-01 23:20:46,559] A new study created in memory with name: no-name-88abe121-551a-47a1-abd1-c919e4d6e9e9
[I 2023-06-01 23:20:46,618] Trial 0 finished with value: 0.9907407407407407 and parameters: {'classifier': 'RandomForest', 'algorithm': 'kd_tree', 'leaf_size': 42, 'metic': 'euclidean'}. Best is trial 0 with value: 0.9907407407407407.
[I 2023-06-01 23:20:46,681] Trial 1 finished with value: 0.9851851851851852 and parameters: {'classifier': 'RandomForest', 'algorithm': 'ball_tree', 'leaf_size': 9, 'metic': 'manhattan'}. Best is trial 0 with value: 0.9907407407407407.
<ipython-input-48-733d150d35b3>:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  c = trial.suggest_loguniform("svc_c", 1e-2, 1e+11)
<ipython-input-48-733d150d35b3>:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This fe

Trial No.=0, HP_Set={'classifier': 'RandomForest', 'algorithm': 'kd_tree', 'leaf_size': 42, 'metic': 'euclidean'}, Score=0.9907407407407407
Best Value =0.9907407407407407
Trial No.=1, HP_Set={'classifier': 'RandomForest', 'algorithm': 'ball_tree', 'leaf_size': 9, 'metic': 'manhattan'}, Score=0.9851851851851852
Best Value =0.9907407407407407


[I 2023-06-01 23:20:46,864] Trial 2 finished with value: 0.08518518518518518 and parameters: {'classifier': 'SVC', 'svc_c': 0.040805008020949295, 'svc_gamma': 0.012204897472741817, 'svc_kernel': 'rbf', 'svc_degree': 6}. Best is trial 0 with value: 0.9907407407407407.


Trial No.=2, HP_Set={'classifier': 'SVC', 'svc_c': 0.040805008020949295, 'svc_gamma': 0.012204897472741817, 'svc_kernel': 'rbf', 'svc_degree': 6}, Score=0.08518518518518518
Best Value =0.9907407407407407
Best trial  accuracy: 0.9907407407407407
parameters for best trail are :
classifier: RandomForest
algorithm: kd_tree
leaf_size: 42
metic: euclidean


### and now a Neural Network

In [49]:
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
import numpy as np

import optuna
from optuna.samplers import TPESampler

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
classes = 10
input_shape = 784
y_train = np_utils.to_categorical(y_train, classes)
y_test = np_utils.to_categorical(y_test, classes)
x_train, y_train, x_test, y_test, input_shape, classes

def log(study, trial):
    print(f"Trial No.={trial.number}, HP_Set={trial.params}, \
          Score={trial.value}")
    print(f"Best Value ={study.best_value}")

def objective_func(trial):
 
    model = Sequential()

    hidden_layer_unit_choice = [32, 64, 256, 512, 1024]

    hidden_layers = trial.suggest_int('hidden_layers', 1, 6)
    
    model.add(Dense(units=trial.suggest_categorical('layer1', [8, 16]), 
                    input_shape=(input_shape, ),
                    name='dense1'))
    model.add(Activation(activation=trial.suggest_categorical(f'activation1',
                                                               ['relu', 'elu'])))

    for i in range(1, hidden_layers):
        
        model.add(Dense(units=trial.suggest_categorical(f'layer{i+1}', 
                                                        hidden_layer_unit_choice)))
        model.add(Dropout(trial.suggest_uniform(f'dropout{i+1}', 0, 0.8)))
        model.add(Activation(activation=trial.suggest_categorical(f'activation{i+1}', 
                                                                  ['relu', 'elu'])))

    model.add(Dense(classes))
    model.add(Activation(activation='softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=trial.suggest_categorical('optimizer', ['rmsprop', 'adam', 'sgd']))

    result = model.fit(x_train, y_train,
                      batch_size=4,
                      epochs=1,
                      verbose=3,
                      validation_split=0.2)

    validation_acc = np.amax(result.history['val_accuracy'])
    print('Validation accuracy:', validation_acc)

    return validation_acc


study = optuna.create_study(direction='maximize', sampler=TPESampler())
# increase the number of trials
study.optimize(objective_func, n_trials=5, callbacks=[log])
best_trial = study.best_trial.value

print(f"Best trial  accuracy: {best_trial}")
print("parameters for best trail are :")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

[I 2023-06-01 23:20:47,150] A new study created in memory with name: no-name-2b21ccd3-bdc2-4a7a-b67c-176f152a3f9c
[I 2023-06-01 23:20:56,176] Trial 0 finished with value: 0.9338333606719971 and parameters: {'hidden_layers': 1, 'layer1': 16, 'activation1': 'relu', 'optimizer': 'adam'}. Best is trial 0 with value: 0.9338333606719971.


Validation accuracy: 0.9338333606719971
Trial No.=0, HP_Set={'hidden_layers': 1, 'layer1': 16, 'activation1': 'relu', 'optimizer': 'adam'},           Score=0.9338333606719971
Best Value =0.9338333606719971


<ipython-input-49-1804c6a8cc52>:46: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  model.add(Dropout(trial.suggest_uniform(f'dropout{i+1}', 0, 0.8)))
[I 2023-06-01 23:21:09,331] Trial 1 finished with value: 0.9224166870117188 and parameters: {'hidden_layers': 3, 'layer1': 16, 'activation1': 'relu', 'layer2': 256, 'dropout2': 0.28041357201665246, 'activation2': 'relu', 'layer3': 256, 'dropout3': 0.4569717270237932, 'activation3': 'relu', 'optimizer': 'rmsprop'}. Best is trial 0 with value: 0.9338333606719971.


Validation accuracy: 0.9224166870117188
Trial No.=1, HP_Set={'hidden_layers': 3, 'layer1': 16, 'activation1': 'relu', 'layer2': 256, 'dropout2': 0.28041357201665246, 'activation2': 'relu', 'layer3': 256, 'dropout3': 0.4569717270237932, 'activation3': 'relu', 'optimizer': 'rmsprop'},           Score=0.9224166870117188
Best Value =0.9338333606719971


[I 2023-06-01 23:21:19,678] Trial 2 finished with value: 0.9226666688919067 and parameters: {'hidden_layers': 2, 'layer1': 16, 'activation1': 'relu', 'layer2': 512, 'dropout2': 0.752987647076444, 'activation2': 'relu', 'optimizer': 'rmsprop'}. Best is trial 0 with value: 0.9338333606719971.


Validation accuracy: 0.9226666688919067
Trial No.=2, HP_Set={'hidden_layers': 2, 'layer1': 16, 'activation1': 'relu', 'layer2': 512, 'dropout2': 0.752987647076444, 'activation2': 'relu', 'optimizer': 'rmsprop'},           Score=0.9226666688919067
Best Value =0.9338333606719971


[I 2023-06-01 23:21:31,311] Trial 3 finished with value: 0.9340000152587891 and parameters: {'hidden_layers': 3, 'layer1': 16, 'activation1': 'elu', 'layer2': 256, 'dropout2': 0.2854440550096142, 'activation2': 'relu', 'layer3': 64, 'dropout3': 0.29623778176314625, 'activation3': 'relu', 'optimizer': 'adam'}. Best is trial 3 with value: 0.9340000152587891.


Validation accuracy: 0.9340000152587891
Trial No.=3, HP_Set={'hidden_layers': 3, 'layer1': 16, 'activation1': 'elu', 'layer2': 256, 'dropout2': 0.2854440550096142, 'activation2': 'relu', 'layer3': 64, 'dropout3': 0.29623778176314625, 'activation3': 'relu', 'optimizer': 'adam'},           Score=0.9340000152587891
Best Value =0.9340000152587891


[I 2023-06-01 23:21:43,495] Trial 4 finished with value: 0.8494166731834412 and parameters: {'hidden_layers': 4, 'layer1': 8, 'activation1': 'relu', 'layer2': 512, 'dropout2': 0.10321320506851475, 'activation2': 'relu', 'layer3': 64, 'dropout3': 0.18473056180576375, 'activation3': 'relu', 'layer4': 64, 'dropout4': 0.5422709361786545, 'activation4': 'relu', 'optimizer': 'rmsprop'}. Best is trial 3 with value: 0.9340000152587891.


Validation accuracy: 0.8494166731834412
Trial No.=4, HP_Set={'hidden_layers': 4, 'layer1': 8, 'activation1': 'relu', 'layer2': 512, 'dropout2': 0.10321320506851475, 'activation2': 'relu', 'layer3': 64, 'dropout3': 0.18473056180576375, 'activation3': 'relu', 'layer4': 64, 'dropout4': 0.5422709361786545, 'activation4': 'relu', 'optimizer': 'rmsprop'},           Score=0.8494166731834412
Best Value =0.9340000152587891
Best trial  accuracy: 0.9340000152587891
parameters for best trail are :
hidden_layers: 3
layer1: 16
activation1: elu
layer2: 256
dropout2: 0.2854440550096142
activation2: relu
layer3: 64
dropout3: 0.29623778176314625
activation3: relu
optimizer: adam


# TPE is one of the best Bayesian Hyperparameter Optimization Algorithm out there, now choose your poison Optuna Or HyperOpt?